In [1]:
spark

In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from io import BytesIO
import re
# import tensorflow as tf
# import cv2

I will use the functionality of Image Datasource of spark to collect and process images in bytes, and then start processing the images for the neural network

In [2]:
images_folders = ['/CA1/Images/Ireland/','/CA1/Images/Honduran/']

In [3]:
images_rdd = spark.sparkContext.binaryFiles(','.join(images_folders))
# images_rdd = spark.sparkContext.binaryFiles('hdfs://172.24.144.178:9000/CA1/Images/Ireland/Ireland_001.jpeg')

In [4]:
images_rdd

org.apache.spark.api.java.JavaPairRDD@3bc5f783

In [5]:
def extract_data(data):
    file_path, image_data = data
#     image = Image.open(BytesIO(image_data))
#     image_array = np.array(image)
    
    file_name = file_path.split('/')[-1]
    file_name_without_ext = file_name.split(".")[0]
    label,name = file_name_without_ext.split('_')[0], file_name_without_ext.split('_')[1]
    
    return name, label, image_data

In [6]:
imageDf = images_rdd.map(lambda x: extract_data(x)).toDF(["Name","Label","Data"])

In [7]:
pandasImagesDF = imageDf.toPandas()

In [8]:
def processImage(data, target_size=(225,225)):
    imgbytes = BytesIO(data)
    image = Image.open(imgbytes)
    resized_img = image.resize(target_size, Image.ANTIALIAS)
    with BytesIO() as output:
        resized_img.save(output, format="PNG")
        new_image = Image.open(output)

        array = np.asarray(image).astype(np.float32) / 255
        return array

In [9]:
pandasImagesDF["Data"] = pandasImagesDF["Data"].apply(lambda x: processImage(x))

In [10]:
pandasImagesDF

,Name,Label,Data
0,001,Ireland,"[[[0.8, 0.67058825, 0.5019608], [0.7764706, 0...."
1,002,Ireland,"[[[0.3764706, 0.4862745, 0.57254905], [0.37647..."
2,003,Ireland,"[[[0.5137255, 0.5176471, 0.5372549], [0.513725..."
3,004,Ireland,"[[[0.28627452, 0.105882354, 0.011764706], [0.3..."
4,005,Ireland,"[[[0.5137255, 0.5529412, 0.5882353], [0.749019..."
5,006,Ireland,"[[[0.29803923, 0.24313726, 0.19215687], [0.298..."
6,007,Ireland,"[[[0.75686276, 0.79607844, 0.7921569], [0.7450..."
7,008,Ireland,"[[[0.7137255, 0.5254902, 0.23529412], [0.71372..."
8,009,Ireland,"[[[0.9529412, 0.9529412, 0.9529412], [0.952941..."
9,010,Ireland,"[[[0.6862745, 0.5803922, 0.49803922], [0.68627..."


In [11]:
pandasImagesDF = pandasImagesDF.sort_values(by=["Name"])

In [13]:
pandasImagesDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    40 non-null     object
 1   Label   40 non-null     object
 2   Data    40 non-null     object
dtypes: object(3)
memory usage: 1.2+ KB


In [ ]:
pandasImagesDF

In [12]:
from platform import python_version

print(python_version())

3.10.12


In [13]:
import matplotlib.pyplot as plt

In [ ]:
np.array(pandasImagesDF["Data"][9]).astype("float32").shape

In [ ]:
# Determine the dimensions of the image
width =  225
height =  225

# Convert the byte array to a numpy array of uint8 data type
image_data = np.array(pandasImagesDF["Data"][0], dtype=np.uint8)
# image_data.reshape(-1,1)
# Reshape the array to match the dimensions of the image
# image_data = image_data.reshape((height, width))

# Create an image from the byte array
image = Image.fromarray(image_data)

# Display the image
image.show()

Importing from Keras functionality necessary to implement CNN

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from tensorflow.keras import utils
from sklearn.metrics import accuracy_score

For Now there is only 20 photos per Category, the training size will be 28 and test 12, This is for testing. After this I'll get more images to improve the CNN

In [18]:
X_train = pandasImagesDF.iloc[:-12,2].values
y_train = pandasImagesDF.iloc[:-12,1].values
X_test = pandasImagesDF.iloc[-12:,2].values
y_test = pandasImagesDF.iloc[-12:,1].values


In [19]:
from sklearn.preprocessing import OneHotEncoder

In [20]:
encoder = OneHotEncoder(sparse_output=False)
y_train = np.array(y_train).reshape(-1, 1)
y_train = encoder.fit_transform(y_train)

y_test = np.array(y_test).reshape(-1,1)
y_test = encoder.fit_transform(y_test)

In [21]:
X_train = X_train.reshape(-1,1)
X_test = X_test.reshape(-1,1)

In [59]:
X_train.reshape(X_train.shape[0],225,225,3)

ValueError: cannot reshape array of size 28 into shape (28,225,225,3)

In [23]:
p = np.reshape(X_train[0][0], (225,225,3))

In [24]:
p.shape

(225, 225, 3)

In [61]:
X_train_reshaped = []
X_test_reshaped = []

for x in X_train:
#     x = x.reshape(-1,1)
    X_train_reshaped.append(x.reshape(225,225,3))
#     print(x[0])
#     print(x[0].shape)

for x in X_test:
#     x = x.reshape(-1,1)
    X_test_reshaped.append(x)
    

In [62]:
# X_train = np.reshape(X_train, (X_train.shape[0], 225,225,1))
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
# y_train = np.reshape(y_train, (y_train.shape[0], 225,225,1))
y_test = np.reshape(y_test, (y_test.shape[0], y_test.shape[1],1))

In [66]:
np.array(X_train_reshaped).shape

(28, 1)

Because the pixels are from 0 to 255, We have to normalize the pixels

In [28]:
# X_train_reshaped/=255
# X_test_reshaped/=255

In [49]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),padding="valid", activation="relu", input_shape=(225,225,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation="relu"))

model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dense(2,activation="softmax"))

/home/jose/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [50]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

In [51]:
x_train = np.stack(X_train_reshaped, axis=0)  
x_test = np.stack(X_test_reshaped,axis=0)

In [52]:
model.fit(X_train_reshaped, y_train, batch_size=14, epochs=10, validation_data=(X_test_reshaped, y_test))

ValueError: Data cardinality is ambiguous. Make sure all arrays contain the same number of samples.'x' sizes: 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1
'y' sizes: 28
